In [1]:
import pandas as pd
from pycaret.classification import *
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_20newsgroups

In [2]:
from pycaret.datasets import get_data
dataset = get_data('juice')

print(dataset.shape)

,Id,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,1,CH,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1
1,2,CH,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1
2,3,CH,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1
3,4,MM,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1
4,5,CH,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0


(1070, 19)


In [3]:
py = setup(
    data=dataset,
    target='STORE',
    session_id=123,
    experiment_name=None,
    # text_features=["Purchase"],
    # ignore_features=["Id"],
    # keep_features=["SalePriceCH"],
    # preprocess=False,
    # imputation_type='simple',
    # numeric_iterative_imputer='lightgbm',
    # categorical_iterative_imputer='lightgbm',
    # max_encoding_ohe=5,
    normalize=True,
    # remove_outliers=True,
    # outliers_method="lof",
    # outliers_threshold=0.5,
    # remove_multicollinearity=True,
    # multicollinearity_threshold=0.8,
    # bin_numeric_features=["PriceMM"],
    # feature_selection=True,
    # feature_selection_method="classic",
    # feature_selection_estimator="rf",
    # n_features_to_select=7,
    # custom_pipeline=[("pca", PCA(n_components=5))],
    system_log=False,
)

In [4]:
a = py._internal_pipeline
print(a)
a.fit(py.X, py.y)
X, y = a.transform(py.X, py.y)
print(X)
print(X.shape)

Pipeline(steps=[('ordinal_encoding',
                 TransfomerWrapper(include=['Purchase', 'Store7'], transformer=OrdinalEncoder(handle_missing='return_nan', mapping=[{'col': 'Purchase', 'mapping': {nan: -1, 'CH': 1, 'MM': 0}}, {'col': 'Store7', 'mapping': {nan: -1, 'No': 1, 'Yes': 0}}]))),
                ('normalize', TransfomerWrapper(transformer=StandardScaler()))])
            Id  Purchase  WeekofPurchase   StoreID   PriceCH   PriceMM  \
0    -0.636165 -1.251378        0.039785  1.317293 -0.072806  0.704191   
1    -0.859551 -1.251378        0.425612 -0.849174 -0.072806  0.704191   
2    -1.490859  0.799119       -1.567828  1.317293 -1.740746 -0.710313   
3    -0.671777  0.799119        0.747134 -1.282468 -0.072806 -0.710313   
4     0.412779  0.799119        0.682830  1.317293 -0.072806  0.331953   
...        ...       ...             ...       ...       ...       ...   
1065 -1.228623  0.799119       -1.374914 -1.282468 -1.740746 -0.710313   
1066  1.306323 -1.251378       -1

In [5]:
best_model = create_model("lr")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9333,0.9968,0.9225,0.9387,0.9346,0.9135,0.9141
1,0.9867,1.0000,0.9857,0.9875,0.9866,0.9827,0.9829
2,0.9467,0.9940,0.9279,0.9517,0.9468,0.9305,0.9314
3,0.9067,0.9930,0.9038,0.9202,0.9070,0.8792,0.8818
4,0.9467,0.9963,0.9368,0.9517,0.9475,0.9307,0.9315
5,0.9600,0.9989,0.9514,0.9598,0.9595,0.9480,0.9482
6,0.9733,0.9961,0.9714,0.9778,0.9735,0.9654,0.9663
7,0.8933,0.9906,0.8743,0.8945,0.8934,0.8615,0.8617
8,0.9865,0.9993,0.9857,0.9873,0.9865,0.9825,0.9827
9,0.9324,0.9905,0.9314,0.9344,0.9320,0.9126,0.9133


In [6]:
best_model

LogisticRegression(max_iter=1000, random_state=123)

In [7]:
py.predict_model(best_model)

KeyError: 'dtypes'